In [21]:
from autoplan.trainer import ParserTrainer, ClassEvaluation, ClassifierTrainer
from autoplan.token import OCamlTokenizer
from autoplan.vis import plot_accuracy, plot_cm, plot_loss
from autoplan.dataset import TrainVal, RandomSplit, build_synthetic_dataset, set_random_seed
from autoplan.generator import ProgramGenerator
from grammars.rainfall.ocaml2 import Program, Labels
from autoplan.parsing import ProgramParser
from autoplan.neighbors import TreeNNClassifier

import pandas as pd
from tqdm.auto import tqdm
from scripts.rainfall_ingest import load_full_t1, ingest_dataset, load_new_labels
import torch
import seaborn as sns
from iterextras import par_for
from pickle_cache import PickleCache
from torch import nn
import numpy as np
import dataclasses
import os

pcache = PickleCache()
device = torch.device('cuda:7')

In [2]:
known_student_dataset = ingest_dataset('T1')
unknown_student_dataset = load_new_labels(known_student_dataset.vocab_index)

Skipped 16 programs


In [7]:
nn_cls = TreeNNClassifier(known_student_dataset, OCamlTokenizer())
nn_unk_prog, nn_unk_ds = nn_cls.compute_programs(unknown_student_dataset)
nn_unk_dist = nn_cls.compute_distance_matrix(nn_unk_prog)

In [15]:
best_nn_eval = nn_cls.eval(nn_unk_ds, nn_unk_dist)

In [11]:
rnn_cls = ClassifierTrainer(
    known_student_dataset, 
    split=TrainVal(known_student_dataset, unknown_student_dataset),
    model_opts={'model': nn.LSTM, 'embedding_size': 128, 'hidden_size': 512},
    device=device)
_1, _2, val_eval, state = rnn_cls.train(epochs=100, progress=True)

In [14]:
best_rnn_eval = val_eval[np.argmax([evl.accuracy for evl in val_eval])]

In [17]:
# from parser_evaluation.ipynb
best_parser_eval = pcache.get('parser_best_eval')

In [18]:
all_evals = {
    'Nearest-neighbors': best_nn_eval,
    'Recurrent neural network': best_rnn_eval,
    'Neural approximate parser': best_parser_eval
}

In [25]:
{k: v.accuracy for k, v in all_evals.items()}

{'Nearest-neighbors': 0.8888888888888888,
 'Recurrent neural network': 0.8461538461538461,
 'Neural approximate parser': 0.8571428571428571}